In [154]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import plotly.io as pio
import polars as pl

In [155]:
data_file_path = os.getenv("EMPLOYMENT_DATA_PATH")

if data_file_path is not None:
    df = pl.read_csv(data_file_path)
else:
    print("Environment variable EMPLOYMENT_DATA_PATH is not set.")

In [156]:
len(df)

824764

In [157]:
df.head()

fipstate,fipscty,naics,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4,censtate,cencty
i64,i64,str,str,i64,str,i64,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64
1,1,"""------""","""G""",11510,"""G""",99870,"""G""",385755,867,"""412""","""200""","""112""","""92""","""38""","""10""","""N""","""N""","""N""","""N""","""N""","""N""","""N""",63,1
1,1,"""11----""","""H""",86,"""H""",1206,"""H""",4979,10,"""7""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""",63,1
1,1,"""113///""","""G""",73,"""H""",1033,"""H""",4210,7,"""5""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""",63,1
1,1,"""22----""","""H""",170,"""H""",5050,"""H""",19912,8,"""N""","""N""","""N""","""3""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""",63,1
1,1,"""221///""","""H""",170,"""H""",5050,"""H""",19912,8,"""N""","""N""","""N""","""3""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""","""N""",63,1


In [158]:
expr_exists = '["N"]' in df.to_pandas().values
print(expr_exists)

False


### Change to "N" values to official null

In [159]:
company_size_ranges = ['n<5', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499', 'n500_999', 'n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4']
for c in company_size_ranges:
    df = df.with_columns(pl.when(pl.col(c) == 'N').then(None).otherwise(pl.col(c)).alias(c))

    df = df.with_columns(pl.col(c).cast(pl.Int32))
    


df.limit(3)

fipstate,fipscty,naics,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,n<5,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4,censtate,cencty
i64,i64,str,str,i64,str,i64,str,i64,i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i64,i64
1,1,"""------""","""G""",11510,"""G""",99870,"""G""",385755,867,412,200,112,92,38,10,null,null,null,null,null,null,null,63,1
1,1,"""11----""","""H""",86,"""H""",1206,"""H""",4979,10,7,null,null,null,null,null,null,null,null,null,null,null,null,63,1
1,1,"""113///""","""G""",73,"""H""",1033,"""H""",4210,7,5,null,null,null,null,null,null,null,null,null,null,null,null,63,1


### Convert State Fips Code to State Name

In [160]:
path = os.getenv("FIPS_FILE_PATH")
conv_df = pl.read_csv(path, separator='\t')
conv_df = conv_df.rename({'StateFIPS': 'fipstate'})
print(len(conv_df))
conv_df = conv_df.filter(pl.col('StateName') != 'District of Columbia')
print(len(conv_df))
conv_df

3143
3142


fipstate,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY,CountyCBSA
i64,i64,str,str,i64,str,str,i64
1,1,"""Autauga""","""Alabama""",1001,"""AL""","""AL | AUTAUGA""",33860
1,3,"""Baldwin""","""Alabama""",1003,"""AL""","""AL | BALDWIN""",19300
1,5,"""Barbour""","""Alabama""",1005,"""AL""","""AL | BARBOUR""",21640
1,7,"""Bibb""","""Alabama""",1007,"""AL""","""AL | BIBB""",13820
1,9,"""Blount""","""Alabama""",1009,"""AL""","""AL | BLOUNT""",13820
1,11,"""Bullock""","""Alabama""",1011,"""AL""","""AL | BULLOCK""",null
1,13,"""Butler""","""Alabama""",1013,"""AL""","""AL | BUTLER""",null
1,15,"""Calhoun""","""Alabama""",1015,"""AL""","""AL | CALHOUN""",11500
1,17,"""Chambers""","""Alabama""",1017,"""AL""","""AL | CHAMBERS""",29300


# Group by State

In [161]:
state_df = df.group_by("fipstate").agg(pl.sum("emp").alias("emp"))

state_df.limit(3)

state_names = conv_df[['fipstate', 'StateAbbr', 'StateName']]

state_names = state_names.group_by('fipstate', 'StateName').agg(pl.first('StateAbbr').alias('StateAbbr')).sort('fipstate')

state_names.limit(3)

fipstate,StateName,StateAbbr
i64,str,str
1,"""Alabama""","""AL"""
2,"""Alaska""","""AK"""
4,"""Arizona""","""AZ"""


In [162]:
combined = state_df.join(other=state_names, on='fipstate', how='inner')

combined.limit(3)

fipstate,emp,StateName,StateAbbr
i64,i64,str,str
6,91087226,"""California""","""CA"""
23,2672548,"""Maine""","""ME"""
12,51509684,"""Florida""","""FL"""


# Population Data

In [163]:
data_file_path = os.getenv("POPULATION_DATA_PATH")

if data_file_path is not None:
    pop_dat = pl.read_csv(data_file_path)
else:
    print("Environment variable POPULATION_DATA_PATH is not set.")
pop_dat.limit(5)

table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts),,_duplicated_0,_duplicated_1,_duplicated_2
str,str,str,str,str
"""Annual Estimat…","""""","""""","""""",""""""
"""Geographic Are…","""April 1, 2020 …","""Population Est…","""""",""""""
"""""","""""","""2020""","""2021""","""2022"""
"""United States""","""331,449,520""","""331,511,512""","""332,031,554""","""333,287,557"""
"""Northeast""","""57,609,156""","""57,448,898""","""57,259,257""","""57,040,406"""


In [164]:
pop_dat = pop_dat.transpose()
pop_dat.limit(3)

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,column_37,column_38,column_39,column_40,column_41,column_42,column_43,column_44,column_45,column_46,column_47,column_48,column_49,column_50,column_51,column_52,column_53,column_54,column_55,column_56,column_57,column_58,column_59,column_60,column_61,column_62,column_63,column_64,column_65
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Annual Estimat…","""Geographic Are…","""""","""United States""","""Northeast""","""Midwest""","""South""","""West""",""".Alabama""",""".Alaska""",""".Arizona""",""".Arkansas""",""".California""",""".Colorado""",""".Connecticut""",""".Delaware""",""".District of C…",""".Florida""",""".Georgia""",""".Hawaii""",""".Idaho""",""".Illinois""",""".Indiana""",""".Iowa""",""".Kansas""",""".Kentucky""",""".Louisiana""",""".Maine""",""".Maryland""",""".Massachusetts…",""".Michigan""",""".Minnesota""",""".Mississippi""",""".Missouri""",""".Montana""",""".Nebraska""",""".Nevada""",""".New Hampshire…",""".New Jersey""",""".New Mexico""",""".New York""",""".North Carolin…",""".North Dakota""",""".Ohio""",""".Oklahoma""",""".Oregon""",""".Pennsylvania""",""".Rhode Island""",""".South Carolin…",""".South Dakota""",""".Tennessee""",""".Texas""",""".Utah""",""".Vermont""",""".Virginia""",""".Washington""",""".West Virginia…",""".Wisconsin""",""".Wyoming""","""""",""".Puerto Rico""","""Note: The esti…","""Suggested Cita…","""Annual Estimat…","""Source: U.S. C…","""Release Date: …"
"""""","""April 1, 2020 …","""""","""331,449,520""","""57,609,156""","""68,985,537""","""126,266,262""","""78,588,565""","""5,024,356""","""733,378""","""7,151,507""","""3,011,555""","""39,538,245""","""5,773,733""","""3,605,942""","""989,957""","""689,546""","""21,538,226""","""10,711,937""","""1,455,273""","""1,839,092""","""12,812,545""","""6,785,668""","""3,190,372""","""2,937,847""","""4,505,893""","""4,657,749""","""1,362,341""","""6,177,213""","""7,029,949""","""10,077,325""","""5,706,504""","""2,961,288""","""6,154,920""","""1,084,197""","""1,961,489""","""3,104,624""","""1,377,518""","""9,289,031""","""2,117,527""","""20,201,230""","""10,439,414""","""779,091""","""11,799,374""","""3,959,346""","""4,237,291""","""13,002,689""","""1,097,371""","""5,118,429""","""886,677""","""6,910,786""","""29,145,428""","""3,271,614""","""643,085""","""8,631,384""","""7,705,247""","""1,793,755""","""5,893,725""","""576,837""","""""","""3,285,874""","""""","""""","""""","""""",""""""
"""""","""Population Est…","""2020""","""331,511,512""","""57,448,898""","""68,961,043""","""126,450,613""","""78,650,958""","""5,031,362""","""732,923""","""7,179,943""","""3,014,195""","""39,501,653""","""5,784,865""","""3,597,362""","""992,114""","""670,868""","""21,589,602""","""10,729,828""","""1,451,043""","""1,849,202""","""12,786,580""","""6,788,799""","""3,190,571""","""2,937,919""","""4,507,445""","""4,651,664""","""1,363,557""","""6,173,205""","""6,995,729""","""10,069,577""","""5,709,852""","""2,958,141""","""6,153,998""","""1,087,075""","""1,962,642""","""3,115,648""","""1,378,587""","""9,271,689""","""2,118,390""","""20,108,296""","""10,449,445""","""779,518""","""11,797,517""","""3,964,912""","""4,244,795""","""12,994,440""","""1,096,345""","""5,131,848""","""887,799""","""6,925,619""","""29,232,474""","""3,283,785""","""642,893""","""8,636,471""","""7,724,031""","""1,791,420""","""5,896,271""","""577,605""","""""","""3,281,557""","""""","""""","""""","""""",""""""


In [165]:
pop_dat = pop_dat.drop('column_0', 'column_1', 'column_59', 'column_61', 'column_62', 'column_63', 'column_64', 'column_65')
pop_dat = pop_dat.rename({'column_2': 'year'})
pop_dat.limit(3)

year,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,column_37,column_38,column_39,column_40,column_41,column_42,column_43,column_44,column_45,column_46,column_47,column_48,column_49,column_50,column_51,column_52,column_53,column_54,column_55,column_56,column_57,column_58,column_60
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""""","""United States""","""Northeast""","""Midwest""","""South""","""West""",""".Alabama""",""".Alaska""",""".Arizona""",""".Arkansas""",""".California""",""".Colorado""",""".Connecticut""",""".Delaware""",""".District of C…",""".Florida""",""".Georgia""",""".Hawaii""",""".Idaho""",""".Illinois""",""".Indiana""",""".Iowa""",""".Kansas""",""".Kentucky""",""".Louisiana""",""".Maine""",""".Maryland""",""".Massachusetts…",""".Michigan""",""".Minnesota""",""".Mississippi""",""".Missouri""",""".Montana""",""".Nebraska""",""".Nevada""",""".New Hampshire…",""".New Jersey""",""".New Mexico""",""".New York""",""".North Carolin…",""".North Dakota""",""".Ohio""",""".Oklahoma""",""".Oregon""",""".Pennsylvania""",""".Rhode Island""",""".South Carolin…",""".South Dakota""",""".Tennessee""",""".Texas""",""".Utah""",""".Vermont""",""".Virginia""",""".Washington""",""".West Virginia…",""".Wisconsin""",""".Wyoming""",""".Puerto Rico"""
"""""","""331,449,520""","""57,609,156""","""68,985,537""","""126,266,262""","""78,588,565""","""5,024,356""","""733,378""","""7,151,507""","""3,011,555""","""39,538,245""","""5,773,733""","""3,605,942""","""989,957""","""689,546""","""21,538,226""","""10,711,937""","""1,455,273""","""1,839,092""","""12,812,545""","""6,785,668""","""3,190,372""","""2,937,847""","""4,505,893""","""4,657,749""","""1,362,341""","""6,177,213""","""7,029,949""","""10,077,325""","""5,706,504""","""2,961,288""","""6,154,920""","""1,084,197""","""1,961,489""","""3,104,624""","""1,377,518""","""9,289,031""","""2,117,527""","""20,201,230""","""10,439,414""","""779,091""","""11,799,374""","""3,959,346""","""4,237,291""","""13,002,689""","""1,097,371""","""5,118,429""","""886,677""","""6,910,786""","""29,145,428""","""3,271,614""","""643,085""","""8,631,384""","""7,705,247""","""1,793,755""","""5,893,725""","""576,837""","""3,285,874"""
"""2020""","""331,511,512""","""57,448,898""","""68,961,043""","""126,450,613""","""78,650,958""","""5,031,362""","""732,923""","""7,179,943""","""3,014,195""","""39,501,653""","""5,784,865""","""3,597,362""","""992,114""","""670,868""","""21,589,602""","""10,729,828""","""1,451,043""","""1,849,202""","""12,786,580""","""6,788,799""","""3,190,571""","""2,937,919""","""4,507,445""","""4,651,664""","""1,363,557""","""6,173,205""","""6,995,729""","""10,069,577""","""5,709,852""","""2,958,141""","""6,153,998""","""1,087,075""","""1,962,642""","""3,115,648""","""1,378,587""","""9,271,689""","""2,118,390""","""20,108,296""","""10,449,445""","""779,518""","""11,797,517""","""3,964,912""","""4,244,795""","""12,994,440""","""1,096,345""","""5,131,848""","""887,799""","""6,925,619""","""29,232,474""","""3,283,785""","""642,893""","""8,636,471""","""7,724,031""","""1,791,420""","""5,896,271""","""577,605""","""3,281,557"""


In [166]:
pop_dat = pop_dat.transpose()
pop_dat.limit(3)

column_0,column_1,column_2,column_3,column_4
str,str,str,str,str
"""""","""""","""2020""","""2021""","""2022"""
"""United States""","""331,449,520""","""331,511,512""","""332,031,554""","""333,287,557"""
"""Northeast""","""57,609,156""","""57,448,898""","""57,259,257""","""57,040,406"""


In [167]:
pop_dat = pop_dat.slice(6, len(pop_dat))
pop_dat.limit(3)

column_0,column_1,column_2,column_3,column_4
str,str,str,str,str
""".Alabama""","""5,024,356""","""5,031,362""","""5,049,846""","""5,074,296"""
""".Alaska""","""733,378""","""732,923""","""734,182""","""733,583"""
""".Arizona""","""7,151,507""","""7,179,943""","""7,264,877""","""7,359,197"""


In [168]:
pop_dat = pop_dat.drop('column_1')\
        .rename({'column_0': 'StateName', 'column_2': 'population_2020', 'column_3': 'population_2021', 'column_4': 'population_2022'})\
        .with_columns(pl.col('StateName').str.replace('.', ''))

pop_dat.limit(3)

StateName,population_2020,population_2021,population_2022
str,str,str,str
"""Alabama""","""5,031,362""","""5,049,846""","""5,074,296"""
"""Alaska""","""732,923""","""734,182""","""733,583"""
"""Arizona""","""7,179,943""","""7,264,877""","""7,359,197"""


In [169]:
pop_dat = pop_dat.with_columns(pl.col('population_2020').str.replace_all(',', '').cast(pl.Int32))\
        .with_columns(pl.col('population_2021').str.replace_all(',', '').cast(pl.Int32))\
        .with_columns(pl.col('population_2022').str.replace_all(',', '').cast(pl.Int32))

                    
pop_dat.limit(3)

StateName,population_2020,population_2021,population_2022
str,i32,i32,i32
"""Alabama""",5031362,5049846,5074296
"""Alaska""",732923,734182,733583
"""Arizona""",7179943,7264877,7359197


# Join Population, Employment, and Fips data

In [170]:
df = combined.join(other=pop_dat, how='inner', on='StateName')
df.limit(3)

fipstate,emp,StateName,StateAbbr,population_2020,population_2021,population_2022
i64,i64,str,str,i32,i32,i32
1,8876890,"""Alabama""","""AL""",5031362,5049846,5074296
2,1353372,"""Alaska""","""AK""",732923,734182,733583
4,15199279,"""Arizona""","""AZ""",7179943,7264877,7359197


In [171]:
df = df.with_columns(pl.col('emp').cast(pl.Int32))\
        .with_columns(emp_ratio_20 = pl.col('population_2020') / pl.col('emp'))\
        .with_columns(emp_ratio_21 = pl.col('population_2021') / pl.col('emp'))\
        .with_columns(emp_ratio_22 = pl.col('population_2022') / pl.col('emp'))
df.limit(3)

fipstate,emp,StateName,StateAbbr,population_2020,population_2021,population_2022,emp_ratio_20,emp_ratio_21,emp_ratio_22
i64,i32,str,str,i32,i32,i32,f64,f64,f64
1,8876890,"""Alabama""","""AL""",5031362,5049846,5074296,0.566793,0.568876,0.57163
2,1353372,"""Alaska""","""AK""",732923,734182,733583,0.541553,0.542484,0.542041
4,15199279,"""Arizona""","""AZ""",7179943,7264877,7359197,0.472387,0.477975,0.484181


# Spatial Employment Map

In [172]:
pio.renderers.default = "browser"
fig = px.choropleth(df, locations="StateAbbr",
                    color="emp_ratio_22",
                    hover_name="StateAbbr",
                    locationmode='USA-states',
                    color_continuous_scale=px.colors.sequential.Oranges,
                    scope="usa")


fig.update_layout(
    title_text="Employment by State",
    geo_scope='usa'
)

fig.show()

In [173]:
data = pl.DataFrame({
    'price':[10, 15, 70],
    'num_products':[20, 17, 40]
})
data = data.with_columns(ratio = data['price'] / data['num_products'])
data

price,num_products,ratio
i64,i64,f64
10,20,0.5
15,17,0.882353
70,40,1.75
